# Monte Carlo Methods in Reinforcement Learning

In this notebook, we explore **Monte Carlo (MC) methods**, which learn from **experience** rather than a complete model of the environment.

Unlike **Dynamic Programming**, MC methods **do not require knowing transition probabilities or rewards** — they rely on **sampling episodes** and **averaging returns** to estimate value functions.

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
- Understand the intuition behind Monte Carlo methods.
- Implement **Monte Carlo prediction** for estimating value functions.
- Apply **Monte Carlo control** to learn optimal policies.
- Distinguish between **First-Visit** and **Every-Visit** Monte Carlo.

## 🧩 What are Monte Carlo Methods?

Monte Carlo methods learn directly from **episodes of experience** by averaging the returns observed following visits to a state or state-action pair.

Key properties:
- Learn from **complete episodes**.
- Require **no knowledge** of transition dynamics.
- Use **sample averages** to estimate expected returns.

### 🔹 Return Definition

For an episode consisting of states, actions, and rewards:

$$
G_t = R_{t+1} + γR_{t+2} + γ^2R_{t+3} + ...
$$

The **value function** is estimated as the average of observed returns following a state:

$$
V(s) = \frac{1}{N(s)} \sum_{i=1}^{N(s)} G_i
$$

## 🧪 Example Environment — Simple Gridworld

We'll simulate a small **gridworld** where an agent starts at a random position and receives +1 when reaching the goal.

In [ ]:
import numpy as np
import random

grid_size = 4
goal_state = (3, 3)
actions = ['up', 'down', 'left', 'right']
gamma = 0.9

def step(state, action):
    i, j = state
    if action == 'up': i = max(i-1, 0)
    elif action == 'down': i = min(i+1, grid_size-1)
    elif action == 'left': j = max(j-1, 0)
    elif action == 'right': j = min(j+1, grid_size-1)
    next_state = (i, j)
    reward = 1 if next_state == goal_state else 0
    done = next_state == goal_state
    return next_state, reward, done

def generate_episode(policy):
    state = (np.random.randint(0, grid_size), np.random.randint(0, grid_size))
    episode = []
    while True:
        action = np.random.choice(actions, p=policy[state])
        next_state, reward, done = step(state, action)
        episode.append((state, action, reward))
        if done:
            break
        state = next_state
    return episode

## 🎯 Monte Carlo Prediction (Estimating V(s))

We estimate the **state-value function** `V(s)` under a given policy by averaging returns from sampled episodes.

In [ ]:
def mc_prediction(policy, episodes=500):
    V = { (i, j): 0 for i in range(grid_size) for j in range(grid_size) }
    returns = { (i, j): [] for i in range(grid_size) for j in range(grid_size) }

    for _ in range(episodes):
        episode = generate_episode(policy)
        G = 0
        visited_states = set()
        for t in reversed(range(len(episode))):
            state, _, reward = episode[t]
            G = gamma * G + reward
            if state not in visited_states:
                returns[state].append(G)
                V[state] = np.mean(returns[state])
                visited_states.add(state)
    return V

# Uniform random policy (equal chance for all actions)
policy = { (i,j): np.ones(len(actions))/len(actions) for i in range(grid_size) for j in range(grid_size) }
V = mc_prediction(policy)
V

✅ The result `V` is the **estimated value** of each state under the random policy.

## ♻️ Monte Carlo Control — Exploring Starts

Monte Carlo **control** aims to find the **optimal policy** without knowing transitions.

We use **Exploring Starts (ES)** — assuming every state-action pair can be visited with non-zero probability.

In [ ]:
def mc_control_es(episodes=5000):
    Q = { (i,j,a): 0 for i in range(grid_size) for j in range(grid_size) for a in actions }
    returns = { (i,j,a): [] for i in range(grid_size) for j in range(grid_size) for a in actions }
    policy = { (i,j): np.ones(len(actions))/len(actions) for i in range(grid_size) for j in range(grid_size) }

    for _ in range(episodes):
        state = (np.random.randint(0, grid_size), np.random.randint(0, grid_size))
        action = np.random.choice(actions)
        episode = []
        next_state, reward, done = step(state, action)
        episode.append((state, action, reward))
        while not done:
            action = np.random.choice(actions, p=policy[next_state])
            new_state, reward, done = step(next_state, action)
            episode.append((next_state, action, reward))
            next_state = new_state

        G = 0
        visited = set()
        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            G = gamma * G + reward
            if (state, action) not in visited:
                returns[(state[0], state[1], action)].append(G)
                Q[(state[0], state[1], action)] = np.mean(returns[(state[0], state[1], action)])
                best_a = np.argmax([Q[(state[0], state[1], a)] for a in actions])
                policy[state] = np.eye(len(actions))[best_a]
                visited.add((state, action))
    return policy, Q

optimal_policy, Q = mc_control_es(episodes=3000)
optimal_policy

✅ The above code demonstrates **Monte Carlo Control with Exploring Starts**, which learns an **optimal policy** purely from sampled episodes — no environment model needed!

## 📘 Summary

- Monte Carlo methods estimate value functions by averaging **sampled returns**.
- They work **without knowing** transition probabilities.
- **MC Prediction** estimates values for a fixed policy.
- **MC Control (Exploring Starts)** finds optimal policies via experience.

Next up: **Temporal Difference (TD) Learning** — combining ideas from **Monte Carlo** and **Dynamic Programming** for faster, online learning!